In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
#import seaborn as sn
from sklearn.model_selection import train_test_split

from random import seed
seed(1)


import tensorflow as tf
from tensorflow import keras
print("Tensorflow Version: ", tf.__version__)
print("Keras Version: ",keras.__version__)


kaggle = 0 # Kaggle path active = 1

# change your local path here
if kaggle == 1 :
    MNIST_PATH= '../input/digit-recognizer'
else:
    MNIST_PATH= '../Digit_Recognition_with_a_Deep_Neural_Network/data/input/digit-recognizer'



import os
for dirname, _, filenames in os.walk(MNIST_PATH): 
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

Tensorflow Version:  2.3.0
Keras Version:  2.4.0
../Digit_Recognition_with_a_Deep_Neural_Network/data/input/digit-recognizer\sample_submission.csv
../Digit_Recognition_with_a_Deep_Neural_Network/data/input/digit-recognizer\test.csv
../Digit_Recognition_with_a_Deep_Neural_Network/data/input/digit-recognizer\train.csv


# Introduction - MNIST Training Competition
Link to the topic: https://www.kaggle.com/c/digit-recognizer/data

This is another Notebook to take a look into annother algorithm. Here I want to give the Deep Neural Network with the Framework Keras a try. As already mentioned in other notebooks, I will skip some explanations about the data set here. Moreover I will use the already discovered knowledge about the data and transform/prepare the data rightaway.

If you are interested in some more clearly analysis of the dataset take a look into my other notebooks about the MNIS-dataset:
- Another MNIST Try: https://www.kaggle.com/skiplik/another-mnist-try
- First NN by Detecting Handwritten Characters: https://www.kaggle.com/skiplik/first-nn-by-detecting-handwritten-characters
...




# Get Data

In [3]:
# Data path and file
#MNIST_PATH= '../input/digit-recognizer'
#MNIST_PATH= '../Another_MNIST_try/data/input/digit-recognizer'
CSV_FILE_TRAIN='train.csv'
CSV_FILE_TEST='test.csv'

def load_mnist_data(minist_path, csv_file):
    csv_path = os.path.join(minist_path, csv_file)
    return pd.read_csv(csv_path)

def load_mnist_data_manuel(minist_path, csv_file):
    csv_path = os.path.join(minist_path, csv_file)
    csv_file = open(csv_path, 'r')
    csv_data = csv_file.readlines()
    csv_file.close()
    return csv_data

def split_train_val(data, val_ratio):
    return 
    

train = load_mnist_data(MNIST_PATH,CSV_FILE_TRAIN)
test = load_mnist_data(MNIST_PATH,CSV_FILE_TEST)

In [4]:
y = train['label'].copy()
X = train.drop(['label'], axis=1)

X_test = test.copy()

## Train / Val Split

In [5]:
print("Shape of the Features: ",X.shape)
print("Shape of the Labels: ", y.shape)

Shape of the Features:  (42000, 784)
Shape of the Labels:  (42000,)


### TK - Label Value Count
Visualizing the label distribution of the full train dataset.

In [6]:
train.value_counts('label')

label
1    4684
7    4401
3    4351
9    4188
2    4177
6    4137
0    4132
4    4072
8    4063
5    3795
dtype: int64

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=43, test_size=0.25, stratify=y)

Comparing the equally splitted train- and val-sets based on the given label y.

In [8]:
print("Train - Set Distribution")
print(y_train.value_counts() / y_train.value_counts().sum() )
print('--------------------------------------------------------------')
print('--------------------------------------------------------------')
print('--------------------------------------------------------------')
print("Val - Set Distribution")
print(y_val.value_counts() / y_val.value_counts().sum() )


Train - Set Distribution
1    0.111524
7    0.104794
3    0.103587
9    0.099714
2    0.099460
6    0.098508
0    0.098381
4    0.096952
8    0.096730
5    0.090349
Name: label, dtype: float64
--------------------------------------------------------------
--------------------------------------------------------------
--------------------------------------------------------------
Val - Set Distribution
1    0.111524
7    0.104762
3    0.103619
9    0.099714
2    0.099429
6    0.098476
0    0.098381
4    0.096952
8    0.096762
5    0.090381
Name: label, dtype: float64


In [9]:
print("X: ", X.shape)
print("X_train: ", X_train.shape)
print("X_val: ", X_val.shape)

print("y_train: ", y_train.shape)
print("y_val: ", y_val.shape)

X:  (42000, 784)
X_train:  (31500, 784)
X_val:  (10500, 784)
y_train:  (31500,)
y_val:  (10500,)


## Building Transforming Piplines

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler

pipeline = Pipeline([
    ('normalizer', Normalizer())
    #('std_scalar',StandardScaler())
])

In [11]:
X_train_prep = pipeline.fit_transform(X_train)      # fitting the pipeline to the train and transform it
X_val_prep = pipeline.transform(X_val)              # transform val data with this information

# Building a Deep Neural Network based on RandomizedSearch

## Preparing Model Visualization with Tensorboard (not for Kaggle)

In [12]:
root_logdir = "../../tensorboard-logs"

print("Relative root_logdir: ",root_logdir)

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir,run_id)

Relative root_logdir:  ../../tensorboard-logs


In [13]:
run_logdir = get_run_logdir()
print("Current run logdir for Tensorboard: ", run_logdir)

Current run logdir for Tensorboard:  ../../tensorboard-logs\run_2021_08_20-08_42_51


In [14]:
run_logdir

'../../tensorboard-logs\\run_2021_08_20-08_42_51'

### Keras Callbacks for Tensorboard
With Keras there is a way of using Callbacks for the Tensorboard to write log files for the board and visualize the different graphs (loss and val curve)


In [15]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

## Building Model Architecture

### Architecture for Hyperparameter Optimization
- Amount of Layers
- Amount of Neurons
- Learningrate
- Checkpoints !!!!!!!!!!!!!!!!!!!!!
- Early Stopping 

In [16]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[784]):
    model = keras.models.Sequential()                               # base model structure (Sequential API by Keras)

    model.add(keras.layers.InputLayer(input_shape=input_shape))     # input layer

    for layer in range(n_hidden):                                   # add layers as often as defined in constructor 
        model.add(keras.layers.Dense(n_neurons,activation="relu"))  # add layer with given neurons and relu activation function

    model.add(keras.layers.Dense(10))                               # add output layer 

    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)   # define optimizer (especially the larning rate for hyperparameter optimization)

    model.compile(loss="mse", optimizer=optimizer)                  # make it ready

    return model


In [17]:
# Using keras wrapper as hull 
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

### Hyperparameter Space

In [18]:
from scipy.stats import reciprocal

# Hyperparameter set
param_dist= {
            "n_neurons": range(20, 300, 20)
            ,"n_hidden": range(10, 150, 10)
            ,"learning_rate": [1e-3, 2e-3, 3e-3]
    }

param_dist_right = {
        "n_neurons": [150]
        ,"n_hidden": [30]
        ,"learning_rate": [2e-3]  
}

## TK - Model Checkpoints

In [19]:
### will be implemented

## Model Training

### Randomized Search

In [20]:
# Finding best hyperparameters with Randomized search
from sklearn.model_selection import RandomizedSearchCV

ran_ker_reg = RandomizedSearchCV(keras_reg, param_dist, n_iter=5, cv=3, random_state=42, return_train_score=True)
history_ker_reg = ran_ker_reg.fit(X_train_prep, y_train, epochs=100, validation_data=(X_val_prep, y_val), callbacks=[keras.callbacks.EarlyStopping(patience=10), tensorboard_cb])

Epoch 1/100
  1/657 [..............................] - ETA: 0s - loss: 29.2500WARNING:tensorflow:From D:\anaconda3\envs\wingpuflake_keras\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
657/657 [==============================] - 4s 6ms/step - loss: 15.8303 - val_loss: 8.3547
Epoch 2/100
657/657 [==============================] - 3s 5ms/step - loss: 8.3384 - val_loss: 8.3401
Epoch 3/100
657/657 [==============================] - 3s 5ms/step - loss: 8.3331 - val_loss: 8.3443
Epoch 4/100
657/657 [==============================] - 3s 5ms/step - loss: 8.3351 - val_loss: 8.3609
Epoch 5/100
657/657 [==============================] - 3s 5ms/step - loss: 8.3328 - val_loss: 8.3610
Epoch 6/100
657/657 [==============================] - 3s 5ms/step - loss: 8.3346 - val_loss: 8.3958
Epoch 7/100
657/657 [=============

In [21]:
history_ker_reg.best_params_

{'n_neurons': 40, 'n_hidden': 60, 'learning_rate': 0.001}

In [22]:
history_ker_reg

RandomizedSearchCV(cv=3,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x000001C177B56610>,
                   n_iter=5,
                   param_distributions={'learning_rate': [0.001, 0.002, 0.003],
                                        'n_hidden': range(10, 150, 10),
                                        'n_neurons': range(20, 300, 20)},
                   random_state=42, return_train_score=True)

# Image Prediction of Unknown Data (Test Data)

## Peparing Test Data

In [23]:
X_test_prep = pipeline.transform(X_test)

In [24]:
X_test_prep

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Creating Competition File

In [25]:
mnist_competition_file = pd.DataFrame(columns=['ImageId','Label'])

## Prediction of Testdata

In [26]:
print("Propability of all lables for given pixels: ", history_ker_reg.predict(X_test_prep[1].reshape(1,-1)))

Propability of all lables for given pixels:  [4.2738686 4.2702904 4.2769136 4.2693253 4.2672048 4.269359  4.2692513
 4.2708354 4.27555   4.268281 ]


In [27]:
print("Predicted Digit: ",np.argmax(history_ker_reg.predict(X_test_prep[1].reshape(1,-1))))

Predicted Digit:  2


In [28]:
i = 1
for row in X_test_prep:
    index = i
    predicted_label = np.argmax(history_ker_reg.predict(row.reshape(1,-1)))

    mnist_competition_file = mnist_competition_file.append({'ImageId': index, 'Label': predicted_label}, ignore_index = True )
    i = i + 1
    pass

In [29]:
mnist_competition_file

,ImageId,Label
0,1,2
1,2,2
2,3,2
3,4,2
4,5,2
...,...,...
27995,27996,2
27996,27997,2
27997,27998,2
27998,27999,2


In [30]:
mnist_competition_file.ImageId = mnist_competition_file.ImageId.astype(int)
mnist_competition_file.Label = mnist_competition_file.Label.astype(int)

In [23]:
mnist_competition_file.to_csv('mnist_submission.csv', index=False)